# 1. 환경 세팅

In [ ]:
import streamlit as st
from langchain_core.messages.chat import ChatMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.prompts import load_prompt
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_teddynote import logging
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
import os

In [17]:
# API KEY 정보로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("[Project] PDF_RAG")

# 캐시 디렉토리 생성
if not os.path.exists(".cache"):
    os.mkdir(".cache")

# 파일 업로드 전용 폴더
if not os.path.exists(".cache/files"):
    os.mkdir(".cache/files")

if not os.path.exists(".cache/embeddings"):
    os.mkdir(".cache/embeddings")

st.title(" AI웹개발자 면접튜터링🚀")

2024-11-23 11:37:35.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:37:35.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


LangSmith 추적을 시작합니다.
[프로젝트명]
[Project] PDF_RAG


DeltaGenerator()

In [33]:
# 처음 1번만 실행하기 위한 코드
if "messages" not in st.session_state:
    # 대화기록을 저장하기 위한 용도로 생성한다.
    st.session_state["messages"] = []

2024-11-23 11:49:58.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [34]:
# 사이드바 생성
with st.sidebar:
    # 초기화 버튼 생성
    clear_btn = st.button("대화 초기화")

    # 파일 업로드
    uploaded_file = st.file_uploader("파일 업로드", type=["pdf"])

2024-11-23 11:49:59.512 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 11:49:59.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [36]:
# 이전 대화를 출력
def print_messages():
    for chat_message in st.session_state["messages"]:
        st.chat_message(chat_message.role).write(chat_message.content)

In [37]:
# 새로운 메시지를 추가
def add_message(role, message):
    st.session_state["messages"].append(ChatMessage(role=role, content=message))

# 2. 문서 로드

In [41]:
# 모델 선정
model = ChatOpenAI(model="gpt-4o-mini")

In [58]:
# 파일을 캐시 저장(시간이 오래 걸리는 작업을 처리할 예정)
@st.cache_resource(show_spinner="업로드한 파일을 처리 중입니다...")
def embed_file(file):
    # 업로드한 파일을 캐시 디렉토리에 저장합니다.
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)

    # 단계 1: 문서 로드(Load Documents)
    loader = PyPDFDirectoryLoader("data/")
    docs = loader.load()

    # 단계 2: 문서 분할(Split Documents)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    split_documents = text_splitter.split_documents(docs)

    # 단계 3: 임베딩(Embedding) 생성
    embeddings = OpenAIEmbeddings()

    # 단계 4: DB 생성(Create DB) 및 저장
    # 벡터스토어를 생성합니다.
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

    # 단계 5: 검색기(Retriever) 생성
    # 문서에 포함되어 있는 정보를 검색하고 생성합니다.
    retriever = vectorstore.as_retriever()
    return retriever


In [61]:
    # 문서의 개수 출력
    print(len(docs))

    # 문서의 내용 출력
    print(docs[7].page_content[:300])

    # metadata 출력
    print(docs[7].metadata)

85
Machine Le ar ning 관 련  면 접  Q&A
8
모델이  참이라고  예측하는  경우는  FPR False Positive Rate, 실제  값이  거짓일  때 ) 과  TPR True Positive Rate, 실제 
값이  참일  때 ) 두  경우로  나뉜다 .
FPR 과  TPR 을  그래프에서  x 축 , y 축으로  동시에  표현한  ROC 커브를  통해  모델이  얼마나  옳은  값을  잘  예측하는지  알  수 
있게  된다 .
ROC 커브가  좌상단과  가까운  경우  좋은  모델이라고  판단할  수 
{'source': 'data\\Machine_Learning관련_면접_QA.pdf', 'page': 7}


In [62]:
# 체인 생성
def create_chain(retriever, model_name="gpt-4o"):
    # 단계 6: 프롬프트 생성(Create Prompt)
    # 프롬프트를 생성합니다.
    prompt = load_prompt("prompts/RAG_project.yaml", encoding="utf-8")

    # 단계 7: 언어모델(LLM) 생성
    # 모델(LLM) 을 생성합니다.
    llm = ChatOpenAI(model_name=model_name, temperature=0)

    # 단계 8: 체인(Chain) 생성
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

In [63]:
# 초기화 버튼이 눌리면...
if clear_btn:
    st.session_state["messages"] = []

# 이전 대화 기록 출력
print_messages()

# 사용자의 입력
user_input = st.chat_input("궁금한 내용을 물어보세요!")

# 경고 메시지를 띄우기 위한 빈 영역
warning_msg = st.empty()

# 만약에 사용자 입력이 들어오면...
if user_input:
    # chain 을 생성
    chain = st.session_state["chain"]

    if chain is not None:
        # 사용자의 입력
        st.chat_message("user").write(user_input)
        # 스트리밍 호출
        response = chain.stream(user_input)
        with st.chat_message("assistant"):
            # 빈 공간(컨테이너)을 만들어서, 여기에 토큰을 스트리밍 출력한다.
            container = st.empty()

            ai_answer = ""
            for token in response:
                ai_answer += token
                container.markdown(ai_answer)

        # 대화기록을 저장한다.
        add_message("user", user_input)
        add_message("assistant", ai_answer)
    else:
        # 파일을 업로드 하라는 경고 메시지 출력
        warning_msg.error("파일을 업로드 해주세요.")

2024-11-23 15:46:08.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 15:46:08.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar